In [1]:
import requests
import os
import pandas as pd

In [2]:
github_api_token = os.getenv("API_KEY_GITHUB")

In [3]:
def parse_owner_repo(url="https://github.com/ivanfioravanti/chatbot-ollama"):
    parts = url.split("/")
    return (parts[3], parts[4]) if len(parts) >= 5 else None 

def get_github_forks_and_stars(github_url, gh_api_token="", dbg_flag=False):
    """Fetch GitHub repo stars and forks counts,
    
    use API token to have higher rate limit
    
    Returns:
        (stars, forks) tuple
        
    Usage:
        >>> github_url = "https://github.com/enricoros/big-AGI/tree/main"
        >>> stars, forks = get_github_forks_and_stars(github_url, gh_api_token=github_api_token)
        >>> print(f"Stars: {stars}\tForks: {forks}\tURL: {github_url}")
        >>> 
    """
    stars = forks = "-1"
    try:
        owner, repo_name = parse_owner_repo(github_url)
        url = f"https://api.github.com/repos/{owner}/{repo_name}"
        if gh_api_token:
            headers = {"Authorization": f"token {gh_api_token}"}
            response = requests.get(url, headers=headers)
        else:
            response = requests.get(url)
        data = response.json()

        stars = data["stargazers_count"]
        forks = data["forks_count"]
        
        if dbg_flag:
            print(f"Stars: {stars}\tForks: {forks}\tURL: {gh_url}")
    except Exception as ex:
        print(f"[ERR] {str(ex)}")
    return stars, forks



def parse_md_url(s):
    import re
    g = re.search(r"\[(.*)\]\((.*)\)", s)
    return (g.group(1),g.group(2)) if g is not None and g.groups() else None

def verify_url(url, url_filters):
    if not url_filters: return True
    res = False
    for i in url_filters:
        if i in url:
            res = True
            break
            
    return res

def parse_md_links(md_str, url_filters=["github.com"]):
    """Parse a markdown string like 
        ### Desktop
        - [Bionic GPT](https://github.com/bionic-gpt/bionic-gpt)
        ### Web
        - [HTML UI](https://github.com/rtcfirefly/ollama-ui)    

    Arguments:
        url_filter (list) - if empty, no filtering on URL domain name

    Returns:
        a dict of dict, e.g.
        {
            "Web & Desktop": {
                "Bionic GPT" : "https://github.com/bionic-gpt/bionic-gpt",
                "HTML UI" : "https://github.com/rtcfirefly/ollama-ui",
            }
        }
    
    """
    resp = {}
    tmp = {}
    title = ""
    for l in md_str.split("\n"):
        l = l.strip()
        if not l: continue
            
        if l.startswith("#"):
            
            if not title:
                title = l.replace("#", "").strip()
            else:
                if tmp:
                    resp[title] = tmp.copy()
                    # reset
                    tmp = {}
                    title = l.replace("#", "").strip()
            continue

        x = parse_md_url(l)
        if x is not None:
            name = x[0]
            url = x[1]
            if not verify_url(url, url_filters): 
                continue
            tmp.update({name:url})
        
    # print(f"l={l}\ntitle={title}\t tmp={tmp}")
        
    if tmp:
        if not title:
            resp = tmp
        else:
            resp[title] = tmp
        
    return resp
                

## Quick Test

In [4]:
# github_url = "https://github.com/BerriAI/litellm"
github_url = "https://github.com/enricoros/big-AGI/tree/main"

stars,forks = get_github_forks_and_stars(github_url, gh_api_token=github_api_token)
stars,forks

(2239, 571)

## Ollama related projects

In [5]:
md_str = """
### Web & Desktop
- [Bionic GPT](https://github.com/bionic-gpt/bionic-gpt)
- [HTML UI](https://github.com/rtcfirefly/ollama-ui)
- [Chatbot UI](https://github.com/ivanfioravanti/chatbot-ollama)
- [Typescript UI](https://github.com/ollama-interface/Ollama-Gui?tab=readme-ov-file)
- [Minimalistic React UI for Ollama Models](https://github.com/richawo/minimal-llm-ui)
- [Web UI](https://github.com/ollama-webui/ollama-webui)
- [Ollamac](https://github.com/kevinhermawan/Ollamac)
- [big-AGI](https://github.com/enricoros/big-agi/blob/main/docs/config-ollama.md)
- [Cheshire Cat assistant framework](https://github.com/cheshire-cat-ai/core)
- [Amica](https://github.com/semperai/amica)
- [chatd](https://github.com/BruceMacD/chatd)
- [Ollama-SwiftUI](https://github.com/kghandour/Ollama-SwiftUI)


### Terminal

- [oterm](https://github.com/ggozad/oterm)
- [Ellama Emacs client](https://github.com/s-kostyaev/ellama)
- [Emacs client](https://github.com/zweifisch/ollama)
- [gen.nvim](https://github.com/David-Kunz/gen.nvim)
- [ollama.nvim](https://github.com/nomnivore/ollama.nvim)
- [ogpt.nvim](https://github.com/huynle/ogpt.nvim)
- [gptel Emacs client](https://github.com/karthink/gptel)
- [Oatmeal](https://github.com/dustinblackman/oatmeal)
- [cmdh](https://github.com/pgibler/cmdh)

### Libraries

- [LangChain](https://github.com/langchain-ai/langchain)
- [LangChainGo](https://github.com/tmc/langchaingo/) 
- [LlamaIndex](https://github.com/run-llama/llama_index)
- [LiteLLM](https://github.com/BerriAI/litellm)
- [OllamaSharp for .NET](https://github.com/awaescher/OllamaSharp)
- [Ollama-rs for Rust](https://github.com/pepperoni21/ollama-rs)
- [Ollama4j for Java](https://github.com/amithkoujalgi/ollama4j)

- [OllamaKit for Swift](https://github.com/kevinhermawan/OllamaKit)
- [Ollama for Dart](https://github.com/breitburg/dart-ollama)
- [Ollama for Laravel](https://github.com/cloudstudio/ollama-laravel)
- [LangChainDart](https://github.com/davidmigloz/langchain_dart)
"""

In [6]:
url_dic = parse_md_links(md_str)

In [7]:
data = []
headers = ["stars","forks", "url", "name", "category"]
for category in url_dic.keys():
    d = url_dic[category]
    for name,url in d.items():
        stars,forks = get_github_forks_and_stars(url, gh_api_token=github_api_token)
        data.append([stars, forks, url, name, category])

In [8]:
df = pd.DataFrame(data, columns=headers)

In [9]:
df

,stars,forks,url,name,category
0,910,80,https://github.com/bionic-gpt/bionic-gpt,Bionic GPT,Web & Desktop
1,305,50,https://github.com/rtcfirefly/ollama-ui,HTML UI,Web & Desktop
2,472,69,https://github.com/ivanfioravanti/chatbot-ollama,Chatbot UI,Web & Desktop
3,136,22,https://github.com/ollama-interface/Ollama-Gui...,Typescript UI,Web & Desktop
4,97,18,https://github.com/richawo/minimal-llm-ui,Minimalistic React UI for Ollama Models,Web & Desktop
5,3080,294,https://github.com/ollama-webui/ollama-webui,Web UI,Web & Desktop
6,261,16,https://github.com/kevinhermawan/Ollamac,Ollamac,Web & Desktop
7,2239,571,https://github.com/enricoros/big-agi/blob/main...,big-AGI,Web & Desktop
8,1458,184,https://github.com/cheshire-cat-ai/core,Cheshire Cat assistant framework,Web & Desktop
9,281,39,https://github.com/semperai/amica,Amica,Web & Desktop


In [10]:
df.to_csv("ollama_ecosystems.csv", index=False)